In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import datetime

import warnings
warnings.filterwarnings('ignore')

# --- Langkah 1: Eksplorasi Data (EDA) & Pemahaman Masalah ---

# Memuat Data (Gunakan nrows untuk membatasi ukuran memori!)
# Ganti 'train.csv' dan 'test.csv' dengan nama file yang sesuai
# Untuk demo, kita gunakan 1 juta baris data training
try:
    train_df = pd.read_csv('train.csv', nrows=1_000_000, parse_dates=['pickup_datetime'])
    test_df = pd.read_csv('test.csv', parse_dates=['pickup_datetime']) # Test set biasanya lebih kecil
except FileNotFoundError:
    print("Pastikan file 'train.csv' dan 'test.csv' ada di direktori yang sama.")
    # Membuat data dummy jika file tidak ditemukan (HANYA UNTUK DEMO CODE AGAR JALAN)
    print("Membuat data dummy...")
    data = {
        'key': [f'key_{i}' for i in range(1000)],
        'fare_amount': np.random.rand(1000) * 50 + 2.5,
        'pickup_datetime': pd.to_datetime(['2010-01-01 12:00:00 UTC'] * 1000) + pd.to_timedelta(np.random.randint(0, 365*24*60*60, 1000), unit='s'),
        'pickup_longitude': np.random.uniform(-74.05, -73.75, 1000),
        'pickup_latitude': np.random.uniform(40.6, 40.9, 1000),
        'dropoff_longitude': np.random.uniform(-74.05, -73.75, 1000),
        'dropoff_latitude': np.random.uniform(40.6, 40.9, 1000),
        'passenger_count': np.random.randint(1, 6, 1000)
    }
    train_df = pd.DataFrame(data)
    test_data = data.copy()
    del test_data['fare_amount'] # Test set tidak punya fare_amount
    test_df = pd.DataFrame(test_data)


print(f"Ukuran data Train awal: {train_df.shape}")
print(f"Ukuran data Test awal: {test_df.shape}")
print("\nContoh data Train:")
print(train_df.head())

# Analisis Target Variable ('fare_amount')
plt.figure(figsize=(10, 6))
sns.histplot(train_df['fare_amount'], bins=100, kde=False)
plt.title('Distribusi Fare Amount (Original)')
plt.xlabel('Fare Amount ($)')
plt.xlim(0, 100) # Batasi x-axis untuk visualisasi yang lebih baik
plt.show()

print("\nStatistik Deskriptif Fare Amount:")
print(train_df['fare_amount'].describe())

# --- Langkah 3 Awal: Pembersihan Data (Sangat Penting di sini!) ---

# 1. Hapus baris dengan fare_amount <= 0
print(f"\nJumlah baris sebelum filter fare_amount: {len(train_df)}")
train_df = train_df[train_df['fare_amount'] > 0]
print(f"Jumlah baris setelah filter fare_amount > 0: {len(train_df)}")

# 2. Hapus baris dengan koordinat tidak valid (di luar batas wajar NYC)
min_lon, max_lon = -75, -72
min_lat, max_lat = 40, 42
train_df = train_df[
    (train_df['pickup_longitude'].between(min_lon, max_lon)) &
    (train_df['pickup_latitude'].between(min_lat, max_lat)) &
    (train_df['dropoff_longitude'].between(min_lon, max_lon)) &
    (train_df['dropoff_latitude'].between(min_lat, max_lat))
]
print(f"Jumlah baris setelah filter koordinat: {len(train_df)}")

# 3. Hapus baris dengan passenger_count aneh (misal > 8 atau < 1)
print(f"\nNilai unik passenger_count sebelum filter: {sorted(train_df['passenger_count'].unique())}")
train_df = train_df[(train_df['passenger_count'] >= 1) & (train_df['passenger_count'] <= 8)]
print(f"Jumlah baris setelah filter passenger_count: {len(train_df)}")

# 4. Hapus baris dengan data hilang (jika ada setelah load)
print(f"\nJumlah NaN sebelum dropna: {train_df.isnull().sum().sum()}")
train_df.dropna(inplace=True)
print(f"Jumlah baris setelah dropna: {len(train_df)}")

# Simpan target variable (setelah cleaning)
y_train = train_df['fare_amount']
# Simpan key test set untuk submission
test_key = test_df['key'] if 'key' in test_df.columns else None # Jika ada kolom 'key'

# Gabungkan train (tanpa target) dan test untuk feature engineering
# Hapus kolom target dari train_df SEBELUM menggabungkan
train_features = train_df.drop(columns=['fare_amount', 'key'], errors='ignore') # Hapus key juga jika ada
test_features = test_df.drop(columns=['key'], errors='ignore')
ntrain = train_features.shape[0]

all_features = pd.concat((train_features, test_features)).reset_index(drop=True)
print(f"\nUkuran data gabungan untuk feature engineering: {all_features.shape}")


# --- Langkah 4: Feature Engineering (Kunci untuk dataset ini!) ---

# Fungsi untuk menghitung jarak Haversine (jarak garis lurus di permukaan bola)
def haversine_distance(lat1, lon1, lat2, lon2):
    p = np.pi/180
    a = 0.5 - np.cos((lat2-lat1)*p)/2 + np.cos(lat1*p) * np.cos(lat2*p) * (1-np.cos((lon2-lon1)*p))/2
    return 12742 * np.arcsin(np.sqrt(a)) # 2*R*arcsin... R=6371 km

# 1. Hitung Jarak Perjalanan
all_features['distance_km'] = haversine_distance(
    all_features['pickup_latitude'], all_features['pickup_longitude'],
    all_features['dropoff_latitude'], all_features['dropoff_longitude']
)

# 2. Ekstrak Fitur dari pickup_datetime
all_features['year'] = all_features['pickup_datetime'].dt.year
all_features['month'] = all_features['pickup_datetime'].dt.month
all_features['day'] = all_features['pickup_datetime'].dt.day
all_features['dayofweek'] = all_features['pickup_datetime'].dt.dayofweek
all_features['hour'] = all_features['pickup_datetime'].dt.hour
# all_features['minute'] = all_features['pickup_datetime'].dt.minute # Mungkin tidak terlalu penting

# 3. Hapus kolom asli yang tidak diperlukan lagi / tidak bisa dipakai model
all_features.drop(['pickup_datetime', 'pickup_longitude', 'pickup_latitude',
                   'dropoff_longitude', 'dropoff_latitude'], axis=1, inplace=True)

print("\nContoh data setelah Feature Engineering:")
print(all_features.head())
print(f"Bentuk data setelah Feature Engineering: {all_features.shape}")


# --- Langkah 3 Akhir: Preprocessing Sisa (Jika Perlu) ---
# Dalam kasus ini, setelah feature engineering, sebagian besar fitur sudah numerik.
# 'passenger_count' bisa dianggap numerik atau kategorikal. Kita biarkan numerik.
# 'year', 'month', 'day', 'dayofweek', 'hour' juga numerik (ordinal).
# Tidak ada encoding yang diperlukan di sini. Scaling mungkin diperlukan jika pakai model linear.


# Pisahkan kembali data train dan test
train_final = all_features[:ntrain]
test_final = all_features[ntrain:]
print(f"\nUkuran data Train Final: {train_final.shape}")
print(f"Ukuran data Test Final: {test_final.shape}")


# --- Langkah 2 & 5: Strategi Validasi & Model Baseline (LightGBM) ---

# Mendefinisikan fungsi evaluasi RMSE
def rmse_cv(model):
    kf = KFold(n_splits=5, shuffle=True, random_state=42).get_n_splits(train_final.values)
    # Gunakan neg_mean_squared_error, lalu akarkan dan balikkan tanda negatif
    rmse = np.sqrt(-cross_val_score(model, train_final.values, y_train,
                                     scoring="neg_mean_squared_error", cv = kf, n_jobs=-1)) # n_jobs=-1 gunakan semua core
    return(rmse)

# Model LightGBM (Parameter awal)
lgbm = lgb.LGBMRegressor(objective='regression_l1', # L1 (MAE) sering lebih robust thd outlier drpd L2 (MSE)
                         metric='rmse', # Monitor RMSE selama training
                         n_estimators=1000, # Jumlah pohon awal, bisa diatur dgn early stopping
                         learning_rate=0.05,
                         num_leaves=31, # Default
                         max_depth=-1, # Default
                         random_state=42,
                         n_jobs=-1)

# Evaluasi dengan Cross-Validation
# Catatan: CV pada 1 juta baris bisa memakan waktu. Anda bisa kurangi n_splits atau data
# Jika Anda hanya ingin menjalankan kode tanpa menunggu lama, komentari bagian CV ini.
print("\nMemulai Cross Validation (bisa memakan waktu)...")
# score = rmse_cv(lgbm)
# print(f"Skor CV (RMSE) LightGBM: {score.mean():.4f} +/- {score.std():.4f}")
# --- Jika CV terlalu lama, lewati saja untuk demo ---
print("Melewati Cross Validation untuk demo cepat.")
# ---

# --- Langkah 6: Hyperparameter Tuning (Dilewati dalam contoh ini) ---
# Gunakan Optuna/GridSearch/RandomSearch di sini.

# --- Langkah 7: Ensembling (Dilewati dalam contoh ini) ---
# Gabungkan prediksi dari LGBM, XGBoost, dll.

# --- Langkah 8: Prediksi Akhir & Submission ---

print("\nMelatih model final pada seluruh data training...")
# Latih model pada seluruh data training (gunakan parameter terbaik jika ada tuning)
# Tambahkan early stopping jika n_estimators besar
# Untuk ini kita perlu validation set kecil atau gunakan fitur early stopping di CV
# Cara sederhana: latih saja dengan n_estimators yang ditentukan
lgbm.fit(train_final.values, y_train)

print("Membuat prediksi pada data test...")
# Membuat prediksi pada data test
predictions = lgbm.predict(test_final.values)
# Pastikan tidak ada prediksi negatif (meskipun L1 harusnya mengurangi ini)
predictions[predictions < 0] = 0

# Membuat file submission
submission = pd.DataFrame()
# Pastikan test_key diambil dengan benar sebelumnya
if test_key is not None:
     submission['key'] = test_key
else:
     # Jika tidak ada 'key' di test set asli, buat index dummy
     submission['key'] = test_final.index
submission['fare_amount'] = predictions
submission.to_csv('submission_taxi.csv', index=False)

print("\nFile submission_taxi.csv telah dibuat.")
print(submission.head())